<a href="https://colab.research.google.com/github/germanrvera/WLG/blob/main/app_optimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pulp pandas

In [18]:
from pulp import *
import collections
import math

def optimizar_cortes_para_un_largo_rollo(largo_rollo_seleccionado, solicitudes_cortes):
    """
    Optimiza el corte de material lineal para un único largo de rollo seleccionado,
    minimizando el número de rollos y el desperdicio.
    Maneja cortes muy grandes por separado para alinear con la expectativa de desperdicio cero
    si el total de metros de estos cortes es un múltiplo del rollo.

    Args:
        largo_rollo_seleccionado (float): La longitud del rollo que se usará para todos los cortes.
        solicitudes_cortes (dict): Un diccionario donde la clave es el largo del corte
                                   (float) y el valor es la cantidad solicitada (int).
                                   Ej: {1.2: 2, 0.8: 3, 2.5: 1, 1.0: 1}
    Returns:
        tuple: (estado_solucion, num_rollos_totales, desperdicio_total, detalles_cortes_por_rollo_generado, advertencias_cortes_grandes)
               - estado_solucion (str): "Optimal", "Infeasible", etc.
               - num_rollos_totales (float): Número total de rollos de 'largo_rollo_seleccionado' utilizados.
               - desperdicio_total (float): Metros totales de desperdicio.
               - detalles_cortes_por_rollo_generado (list of dict): Lista con el detalle de los cortes para cada rollo generado.
               - advertencias_cortes_grandes (list): Lista de diccionarios con info de cortes que excedieron el largo del rollo seleccionado.
    """

    # --- 1. Pre-procesar solicitudes: Separar cortes que exceden el rollo seleccionado ---
    cortes_para_optimizar = {}
    cortes_grandes_externos = [] # Cortes que son mayores que el largo del rollo seleccionado

    for largo, cantidad in solicitudes_cortes.items():
        if largo > largo_rollo_seleccionado:
            cortes_grandes_externos.append({"largo": largo, "cantidad": cantidad})
        else:
            cortes_para_optimizar[largo] = cortes_para_optimizar.get(largo, 0) + cantidad

    # --- Procesar cortes grandes externos: Calculamos su contribución a rollos y desperdicio ---
    total_rollos_grandes_externas = 0
    total_desperdicio_grandes_externas = 0
    detalles_cortes_grandes_externos_formateados = []

    if cortes_grandes_externos:
        # Calcular el total de metros que suman todos los cortes grandes solicitados
        total_metros_requeridos_grandes = sum(c['largo'] * c['cantidad'] for c in cortes_grandes_externos)

        # Calcular cuántos rollos de 'largo_rollo_seleccionado' se necesitan para ESTE TOTAL
        num_rollos_para_total_grandes = math.ceil(total_metros_requeridos_grandes / largo_rollo_seleccionado)

        material_consumido_para_grandes = num_rollos_para_total_grandes * largo_rollo_seleccionado
        desperdicio_para_total_grandes = material_consumido_para_grandes - total_metros_requeridos_grandes

        total_rollos_grandes_externas = num_rollos_para_total_grandes
        total_desperdicio_grandes_externas = desperdicio_para_total_grandes

        # Para el detalle, creamos una entrada consolidada
        detalles_cortes_grandes_externos_formateados.append({
            "Rollo_ID": "RESUMEN_PIEZAS_GRANDES",
            "Tipo_Rollo": largo_rollo_seleccionado,
            "Cortes_en_rollo": [f"TOTAL {total_metros_requeridos_grandes:.1f}m (para {sum(c['cantidad'] for c in cortes_grandes_externos)} piezas grandes > {largo_rollo_seleccionado:.1f}m)"],
            "Desperdicio_en_rollo": desperdicio_para_total_grandes,
            "Metros_Consumidos_para_esta_pieza": total_metros_requeridos_grandes, # Material útil
            "Rollos_Fisicos_Asignados": num_rollos_para_total_grandes # Indicar los rollos que se usan
        })
        # Incluimos cada solicitud de corte grande como una advertencia para mayor detalle
        # pero ya no la usamos para generar "rollos" individuales con desperdicio aquí.

    # --- Si no hay cortes para optimizar con PuLP (solo había grandes), devolver temprano ---
    if not cortes_para_optimizar:
        return "Optimal (Solo Cortes Mayores al Rollo Seleccionado)", \
               total_rollos_grandes_externas, \
               total_desperdicio_grandes_externas, \
               detalles_cortes_grandes_externos_formateados, \
               cortes_grandes_externos # Mantener la lista de solicitudes individuales como advertencia


    # --- 2. Generar patrones de corte válidos para el largo de rollo seleccionado (solo para cortes_para_optimizar) ---
    largos_unicos_a_optimizar = sorted(list(cortes_para_optimizar.keys()), reverse=True)

    def generar_todos_los_patrones(largos_disponibles, largo_maximo_patron, current_pattern=[]):
        patrones = []
        suma_actual = sum(current_pattern)

        if suma_actual <= largo_maximo_patron and current_pattern:
            patrones.append(current_pattern)

        for i, largo in enumerate(largos_disponibles):
            if suma_actual + largo <= largo_maximo_patron:
                nuevos_patrones = generar_todos_los_patrones(
                    largos_disponibles[i:], largo_maximo_patron, current_pattern + [largo]
                )
                patrones.extend(nuevos_patrones)
        return patrones

    todos_los_patrones = [
        tuple(sorted(p)) for p in generar_todos_los_patrones(largos_unicos_a_optimizar, largo_rollo_seleccionado)
    ]
    patrones_unicos = list(collections.OrderedDict.fromkeys(todos_los_patrones))

    if not patrones_unicos:
        # Esto ocurre si no hay cortes 'pequeños' o si los 'pequeños' son tan pequeños que no se forman patrones.
        # Si hay cortes grandes, ya se han procesado arriba.
        # Si no hay ninguno, el resultado será 0 rollos y 0 desperdicio.
        return "No hay patrones válidos generados para cortes pequeños", \
               total_rollos_grandes_externas, \
               total_desperdicio_grandes_externas, \
               detalles_cortes_grandes_externos_formateados, \
               cortes_grandes_externos

    # --- 3. Crear el modelo de optimización (Problema de Programación Lineal) ---
    problema = LpProblem("Minimizar Desperdicio de Corte", LpMinimize)

    # Variables de decisión: x[i] = cuántas veces usamos el patrón i
    x = LpVariable.dicts("UsoPatron", range(len(patrones_unicos)), 0, None, LpInteger)

    # --- 4. Función Objetivo: Minimizar el número total de rollos utilizados ---
    problema += lpSum([x[i] for i in range(len(patrones_unicos))]), "Total de Rollos Usados"

    # --- 5. Restricciones: Asegurarse de que todos los cortes solicitados se cumplan ---
    for largo_requerido, cantidad_solicitada in cortes_para_optimizar.items():
        problema += lpSum([
            x[i] * patrones_unicos[i].count(largo_requerido)
            for i in range(len(patrones_unicos))
        ]) >= cantidad_solicitada, f"Cumplir_Corte_{largo_requerido}"

    # --- 6. Resolver el problema ---
    problema.solve()

    # --- 7. Procesar y devolver los resultados ---
    estado_solucion = LpStatus[problema.status]

    num_rollos_optimizador = 0
    desperdicio_optimizador = 0
    detalles_cortes_optimizador_formateados = []

    if estado_solucion == 'Optimal':
        num_rollos_optimizador = problema.objective.value()

        total_cortado_necesario_optimizador = sum(l * c for l, c in cortes_para_optimizar.items())
        desperdicio_optimizador = (num_rollos_optimizador * largo_rollo_seleccionado) - total_cortado_necesario_optimizador

        rollo_id_contador_opt = 1
        for i in range(len(patrones_unicos)):
            num_usos_patron = int(x[i].varValue)
            if num_usos_patron > 0:
                for _ in range(num_usos_patron):
                    patron_actual = list(patrones_unicos[i])
                    uso_material_en_patron = sum(patron_actual)
                    desperdicio_en_este_rollo = largo_rollo_seleccionado - uso_material_en_patron

                    detalles_cortes_optimizador_formateados.append({
                        "Rollo_ID": f"Opt-Rollo-{rollo_id_contador_opt}",
                        "Tipo_Rollo": largo_rollo_seleccionado,
                        "Cortes_en_rollo": patron_actual,
                        "Desperdicio_en_rollo": desperdicio_en_este_rollo,
                        "Metros_Consumidos_en_este_rollo": uso_material_en_patron
                    })
                    rollo_id_contador_opt += 1

    # Consolidar todos los resultados (cortes grandes externos + optimizados)
    num_rollos_totales_final = num_rollos_optimizador + total_rollos_grandes_externas
    desperdicio_total_final = desperdicio_optimizador + total_desperdicio_grandes_externas

    # Unir los detalles de los rollos de ambos orígenes
    todos_los_detalles_de_rollos = detalles_cortes_grandes_externos_formateados + detalles_cortes_optimizador_formateados

    return estado_solucion, num_rollos_totales_final, desperdicio_total_final, todos_los_detalles_de_rollos, cortes_grandes_externos


# --- EJEMPLO DE USO CON SELECCIÓN DE ROLLO Y ENTRADA INTERACTIVA ---
if __name__ == "__main__":
    ROLLOS_DISPONIBLES = [5.0, 10.0, 40.0] # Los tipos de rollo que podés seleccionar

    print("--- Optimizador de Cortes de Material ---")
    print("Por favor, selecciona el largo del rollo que vas a usar para este cálculo:")
    print(f"Opciones disponibles: {ROLLOS_DISPONIBLES} metros.")

    largo_rollo_seleccionado = 0.0
    while largo_rollo_seleccionado not in ROLLOS_DISPONIBLES:
        try:
            entrada_largo = float(input(f"Ingresa el largo del rollo ({'/'.join(map(str, ROLLOS_DISPONIBLES))}): ").strip())
            if entrada_largo in ROLLOS_DISPONIBLES:
                largo_rollo_seleccionado = entrada_largo
            else:
                print("Largo de rollo no válido. Por favor, elige una de las opciones disponibles.")
        except ValueError:
            print("Entrada no válida. Por favor, ingresa un número.")

    print(f"\nHas seleccionado rollos de {largo_rollo_seleccionado:.1f} metros.")
    print("Ahora, ingresa los cortes solicitados. Formato: [Largo] [Cantidad] (ej: 1.2 5).")
    print("Escriba 'fin' para terminar el ingreso.")

    solicitudes_cortes_ingresadas = {}

    while True:
        entrada = input("Corte (Largo Cantidad) o 'fin': ").strip().lower()
        if entrada == 'fin':
            break
        try:
            largo_str, cantidad_str = entrada.split()
            largo = float(largo_str)
            cantidad = int(cantidad_str)

            if largo <= 0 or cantidad <= 0:
                print("Error: Largo y cantidad deben ser números positivos. Intente de nuevo.")
                continue
            solicitudes_cortes_ingresadas[largo] = solicitudes_cortes_ingresadas.get(largo, 0) + cantidad

        except ValueError:
            print("Error: Formato incorrecto. Por favor, ingresa el largo y la cantidad separados por un espacio (ej: 1.5 3).")
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}")

    if not solicitudes_cortes_ingresadas:
        print("\nNo se ingresaron cortes. No hay nada que optimizar. Saliendo.")
        exit()

    estado, num_rollos_totales, desperdicio_total, detalles_cortes_por_rollo, advertencias_cortes_grandes = \
        optimizar_cortes_para_un_largo_rollo(largo_rollo_seleccionado, solicitudes_cortes_ingresadas)

    print(f"\n--- Resumen Final de la Optimización ---")
    print(f"Largo de rollo seleccionado para el cálculo: {largo_rollo_seleccionado:.1f} metros")
    print(f"Estado de la solución: {estado}")

    if estado in ['Optimal', 'Optimal (Solo Cortes Mayores al Rollo Seleccionado)', 'No hay patrones válidos generados para cortes pequeños']:
        print(f"Número TOTAL de rollos de {largo_rollo_seleccionado:.1f}m necesarios: {num_rollos_totales:.2f} unidades")
        print(f"Desperdicio TOTAL de material: {desperdicio_total:.2f} metros")

        if advertencias_cortes_grandes:
            print("\n--- ¡INFORMACIÓN IMPORTANTE SOBRE CORTES GRANDES! ---")
            print("Los siguientes cortes individuales son más largos que el rollo de material seleccionado.")
            print("Esto significa que cada una de estas piezas finales se formará UNINDO segmentos de varios rollos.")
            print("El cálculo de rollos y desperdicio ya considera la suma total de estos cortes grandes.")
            for adv in advertencias_cortes_grandes:
                print(f"  - Solicitud: {adv['cantidad']}x de {adv['largo']:.1f}m.")


        print("\n--- Detalle de cómo se usarán los rollos ---")
        print("Cada línea representa un rollo físico de 5.0m y cómo se cortará.")
        if detalles_cortes_por_rollo:
            detalles_cortes_por_rollo.sort(key=lambda x: (x.get('Tipo_Rollo', 0), x.get('Rollo_ID', '')))

            for rollo_info in detalles_cortes_por_rollo:
                tipo_rollo = rollo_info["Tipo_Rollo"]
                cortes = rollo_info["Cortes_en_rollo"]
                desperdicio_rollo = rollo_info["Desperdicio_en_rollo"]
                metros_consumidos = rollo_info.get("Metros_Consumidos_en_este_rollo", tipo_rollo - desperdicio_rollo) # Toma el valor guardado o calcula

                # Ajuste para mostrar mejor los cortes grandes en el detalle del rollo
                if "RESUMEN_PIEZAS_GRANDES" in rollo_info["Rollo_ID"]:
                    print(f"  {rollo_info['Rollo_ID']} (Tipo Rollo: {tipo_rollo:.1f}m): {cortes[0]} (Rollos físicos asignados: {rollo_info['Rollos_Fisicos_Asignados']:.2f}, Desperdicio para estas piezas: {desperdicio_rollo:.2f}m)")
                else:
                    print(f"  {rollo_info['Rollo_ID']} (Tipo Rollo: {tipo_rollo:.1f}m): Cortes {cortes} (Usado: {metros_consumidos:.2f}m, Desperdicio en este rollo: {desperdicio_rollo:.2f}m)")
        else:
            print("  No se generaron detalles de cortes por rollo.")

    elif estado == 'Infeasible':
        print("\nLa solución es INFACTIBLE.")
        print("No es posible cumplir con todos los cortes solicitados usando rollos de este largo.")
        print("Esto puede ocurrir si la suma total de material solicitado (incluyendo cortes grandes y pequeños) excede lo que un número razonable de rollos puede proveer, o si no hay patrones de corte válidos.")
        if advertencias_cortes_grandes:
            print("\nConsidera que los siguientes cortes individuales son más grandes que el rollo seleccionado:")
            for corte_grande_info in advertencias_cortes_grandes:
                print(f"  - Solicitud: {corte_grande_info['cantidad']}x de {corte_grande_info['largo']:.1f}m.")
    else:
        print(f"No se pudo encontrar una solución óptima para los cortes solicitados. Estado del optimizador: {estado}")
        print("Por favor, revisa tus entradas o la longitud del rollo seleccionado.")

--- Optimizador de Cortes de Material ---
Por favor, selecciona el largo del rollo que vas a usar para este cálculo:
Opciones disponibles: [5.0, 10.0, 40.0] metros.
Ingresa el largo del rollo (5.0/10.0/40.0): 40

Has seleccionado rollos de 40.0 metros.
Ahora, ingresa los cortes solicitados. Formato: [Largo] [Cantidad] (ej: 1.2 5).
Escriba 'fin' para terminar el ingreso.
Corte (Largo Cantidad) o 'fin': 100 2
Corte (Largo Cantidad) o 'fin': fin

--- Resumen Final de la Optimización ---
Largo de rollo seleccionado para el cálculo: 40.0 metros
Estado de la solución: Optimal (Solo Cortes Mayores al Rollo Seleccionado)
Número TOTAL de rollos de 40.0m necesarios: 5.00 unidades
Desperdicio TOTAL de material: 0.00 metros

--- ¡INFORMACIÓN IMPORTANTE SOBRE CORTES GRANDES! ---
Los siguientes cortes individuales son más largos que el rollo de material seleccionado.
Esto significa que cada una de estas piezas finales se formará UNINDO segmentos de varios rollos.
El cálculo de rollos y desperdicio y